# Scrap My Prop

### Laboratórios de Engenharia Informática

#### "Development of an IT solution for the extraction and automatic analysis of data and relevant information for the calculation of land and properties."

## Importar Bibliotecas Python

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from scipy import stats
from collections import defaultdict

# 'ERA'

## Ler Dados do CSV

In [ ]:
data_era = pd.read_csv('dados/dados_era_01_04.csv')
pd.set_option('display.max_columns', None)
data_era.head()

## Pré-processamento comum a todo o Dataset

#### Funções auxiliares

In [ ]:
def ajustPreco(string):
    res = re.sub(r"(.*?)€",r"\1",string)
    res = res.replace(".", "")
    res = pd.to_numeric(res, errors='ignore')
    return res

#### Colunas 'Distrito' e 'Concelho': Retirar colunas de forma total porque já era uma pré-condição do projeto, mas primeiro verificar se, de facto, todos os campos estão preenchidos com 'Braga';

In [ ]:
print("Distritos existentes:", data_era['Distrito'].unique())
print("Concelhos existentes:", data_era['Concelho'].unique())

In [ ]:
if 'Distrito' in data_era.columns:
    data_era = data_era.drop(['Distrito'], axis = 1) 
if 'Concelho' in data_era.columns:
    data_era = data_era.drop(['Concelho'], axis = 1) 

#### Colunas 'Nome' e 'Referência': Não são características dos imóveis mas sim identificadores, portanto não devem entrar nos cálculos (Drop);

In [ ]:
if 'Nome' in data_era.columns:
    data_era = data_era.drop(['Nome'], axis = 1) 
if 'Referência' in data_era.columns:
    data_era = data_era.drop(['Referência'], axis = 1) 
data_era.info()

#### Coluna 'Preço de Venda': Drop dos imóveis que estão 'Sob Consulta'; Retirar símbolo '€'; Transformar em dado numérico;

In [ ]:
data_era = data_era.drop(data_era[(data_era['Preço de Venda'] == 'Sob Consulta')].index)
data_era.index = np.arange(1, len(data_era) + 1)
data_era['Preço de Venda'] = data_era['Preço de Venda'].apply(ajustPreco)
data_era.head()

#### Coluna 'Freguesia': Não podem existir valores nulos nesta coluna, senão estamos a perder o principal fator de diferenciação dos preços;

In [ ]:
data_era['Freguesia'].isnull().values.any()

#### Coluna 'Estado': 'N/D' deve passar a notação que seja interpretada pelas bibliotecas do Python ('NaN')

In [ ]:
data_era = data_era.replace('N/D',np.nan)
data_era.head()

In [ ]:
data_era.info()

#### Estruturar dados que ainda estão em bruto

In [ ]:
data_era['Finalidade'].unique()

In [ ]:
finalidade = pd.get_dummies(data_era['Finalidade'])
data_era = data_era.drop('Finalidade',axis = 1)
data_era = data_era.join(finalidade)
data_era = data_era.rename(columns={'Venda': 'Finalidade - Venda', 'Venda, Arrendamento': 'Finalidade - Venda Arrendamento'})
data_era.head()

## Pré-processamento diferenciado por tipologia

#### Listar todas as tipologias existentes

In [ ]:
data_era['Tipo de Imóvel'].unique()

#### Dividir os dados por tipologia

In [ ]:
data_era_moradias = data_era.loc[(data_era['Tipo de Imóvel'] == 'Moradia em Banda') | (data_era['Tipo de Imóvel'] == 'Moradia Geminada') | (data_era['Tipo de Imóvel'] == 'Moradia Isolada') | (data_era['Tipo de Imóvel'] == 'Moradia') | (data_era['Tipo de Imóvel'] == 'Andar Moradia') | (data_era['Tipo de Imóvel'] == 'Moradia Rústica')]
data_era_moradias.index = np.arange(1, len(data_era_moradias) + 1)

data_era_apartamentos = data_era.loc[(data_era['Tipo de Imóvel'] == 'Apartamento') | (data_era['Tipo de Imóvel'] == 'Duplex')]
data_era_apartamentos.index = np.arange(1, len(data_era_apartamentos) + 1)

data_era_terrenos = data_era.loc[(data_era['Tipo de Imóvel'] == 'Lote') | (data_era['Tipo de Imóvel'] == 'Terreno Para Construção') | (data_era['Tipo de Imóvel'] == 'Terreno com ruina') | (data_era['Tipo de Imóvel'] == 'Terreno Rústico') | (data_era['Tipo de Imóvel'] == 'Lote Industrial') | (data_era['Tipo de Imóvel'] == 'Terreno') | (data_era['Tipo de Imóvel'] == 'Loteamento') | (data_era['Tipo de Imóvel'] == 'Terreno Misto')]
data_era_terrenos.index = np.arange(1, len(data_era_terrenos) + 1)

data_era_lojas = data_era.loc[(data_era['Tipo de Imóvel'] == 'Loja')]
data_era_lojas.index = np.arange(1, len(data_era_lojas) + 1)

# faltam tipologias

## Dados - Moradias

In [ ]:
data_era_moradias.head()

In [ ]:
data_era_moradias.info()

#### Análise exploratória de dados

In [ ]:
fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
data_era_moradias['Preço de Venda'].hist(bins=30)
plt.xlabel('Preço de Venda')

In [ ]:
fig = plt.figure(figsize = (10,3))
sns.boxplot(x=data_era_moradias['Preço de Venda'])

In [ ]:
fig = plt.figure(figsize = (25,10))

ax1 = fig.add_subplot(2,3,1)
ax1.scatter(data_era_moradias['Área Útil'], data_era_moradias['Preço de Venda'])
ax1.set_xlabel('Área Útil')
ax1.set_ylabel('Preço de Venda')

ax2 = fig.add_subplot(2,3,2)
ax2.scatter(data_era_moradias['Área Terreno'], data_era_moradias['Preço de Venda'])
ax2.set_xlabel('Área Terreno')
ax2.set_ylabel('Preço de Venda')

ax3 = fig.add_subplot(2,3,4)
ax3.scatter(data_era_moradias['Nº Quartos'], data_era_moradias['Preço de Venda'])
ax3.set_xlabel('Nº Quartos')
ax3.set_ylabel('Preço de Venda')


In [ ]:
#cat_data = data_era_moradias.select_dtypes(include=['object'])
cat_data = data_era_moradias[['Freguesia']]

In [ ]:
for cat in cat_data.dtypes[:10].index.values:
    plt.figure(figsize=(20, 8))
    plt.xticks(rotation=90)
    sns.boxplot(x=cat, y='Preço de Venda', data=data_era_moradias) 
    sns.swarmplot(x=cat, y='Preço de Venda', data=data_era_moradias)
    plt.show()

#### Remoção de Outliers

In [ ]:
#outliers = data_era_moradias.loc[data_era_moradias['Preço de Venda'] > 500000].index
#data_era_moradias = data_era_moradias.drop(outliers)
#data_era_moradias.index = np.arange(1, len(data_era_moradias) + 1)
#data_era_moradias.shape

In [ ]:
fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
data_era_moradias['Preço de Venda'].hist(bins=30)
plt.xlabel('Preço de Venda')

In [ ]:
z = np.abs(stats.zscore(data_era_moradias[['Preço de Venda']]))
threshold = 3
print(np.where(z > threshold))
# The first array contains the list of row numbers and second array respective column numbers
#data_era_moradias.loc[ 31 , : ]

In [ ]:
# Só fazer 1 vez
data = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]
data.index = np.arange(1, len(data) + 1)
data

#### Dados em falta por coluna

In [ ]:
percent_missing = data_era_moradias.isnull().sum() * 100 / len(data_era_moradias)
missing_value_data_era_moradias_columns = pd.DataFrame({'percent_missing (%)': percent_missing})
sort_data = missing_value_data_era_moradias_columns.copy()
sort_data.sort_values('percent_missing (%)', inplace=True, ascending=False)
sort_data

#### Remover colunas com percentagem de dados em falta superior a 50 %

In [ ]:
a_manter = list(missing_value_data_era_moradias_columns.index[missing_value_data_era_moradias_columns['percent_missing (%)'] < 50])
data_era_moradias = data_era_moradias[a_manter]
data_era_moradias.head()

#### Dados em falta por linha

In [ ]:
#percent_missing = (1 - data_era_moradias.apply(lambda x: x.count(), axis=1) / len(data_era_moradias.columns)) * 100
#missing_value_data_era_moradias_rows = pd.DataFrame({'percent_missing (%)': percent_missing})
#sort_data = missing_value_data_era_moradias_rows.copy()
#sort_data.sort_values('percent_missing (%)', inplace=True, ascending=False)
#sort_data

#### Estruturar dados que ainda estão em bruto

In [ ]:
data_era_moradias['Zona'].unique()

In [ ]:
def getAcessos(string):
    return re.search(r"Acessos:\s(.*?);",string)

def getCentralidades(string):
    return re.search(r"Centralidade:\s(.*?);",string)

def getProximidades(string):
    return re.search(r"Proximidade:\s(.*?);",string)

In [ ]:
zona = data_era_moradias['Zona']

acessosDict = defaultdict(list)
centralidadesDict = defaultdict(list)
proximidadesDict = defaultdict(list)


j = 0

for value in zona:
    j = j+1
    if "Acessos" in str(value):
        acessos = getAcessos(value)
        if acessos is not None:
            for acesso in acessos.groups():
                acesso = acesso.split(', ')
                for x in acesso:
                    acessosDict[x].append(j)

    if "Centralidade" in str(value):
        centralidades = getCentralidades(value)
        if centralidades is not None:
             for centralidade in centralidades.groups():
                centralidade = centralidade.split(', ')
                for x in centralidade:
                    centralidadesDict[x].append(j)
           
    if "Proximidade" in str(value):
        proximidades = getProximidades(value)
        if proximidades is not None:
            for proximidade in proximidades.groups():
                proximidade = proximidade.split(', ')
                for x in proximidade:
                    proximidadesDict[x].append(j)

In [ ]:
data_era_moradias = data_era_moradias.drop('Zona',axis = 1)

for k, v in acessosDict.items():
    data_era_moradias[k] = 0

for k, v in acessosDict.items():
    for j in v:
        data_era_moradias.at[j, k] = 1
        
data_era_moradias = data_era_moradias.rename(columns={'Calçada': 'Zona Acessos - Calçada', \
                                    'Alcatrão': 'Zona Acessos - Alcatrão', \
                                    'Bons': 'Zona Acessos - Bons', \
                                    'Transportes Púb.': 'Zona Acessos - Transportes Púb.', \
                                    'Auto-Estrada': 'Zona Acessos - Auto-Estrada', \
                                    'Comboio': 'Zona Acessos - Comboio', \
                                    'Terra Batida': 'Zona Acessos - Terra Batida', \
                                    'Aeroporto': 'Zona Acessos - Aeroporto', \
                                    'Metro': 'Zona Acessos - Metro'})

for k, v in centralidadesDict.items():
    data_era_moradias[k] = 0

for k, v in centralidadesDict.items():
    for j in v:
        data_era_moradias.at[j, k] = 1        

data_era_moradias = data_era_moradias.rename(columns={'Distante Cidade': 'Zona Centralidade - Distante Cidade',\
                                   'Periferia Cidade': 'Zona Centralidade - Periferia Cidade', \
                                   'Centro da Cidade': 'Zona Centralidade - Centro da Cidade', \
                                   'Rural': 'Zona Centralidade - Rural'})

for k, v in proximidadesDict.items():
    data_era_moradias[k] = 0

for k, v in proximidadesDict.items():
    for j in v:
        data_era_moradias.at[j, k] = 1

data_era_moradias = data_era_moradias.rename(columns={'Centros Comerciais': 'Zona Proximidade - Centros Comerciais',\
                                   'Escolas': 'Zona Proximidade - Escolas',\
                                   'Ginásio': 'Zona Proximidade - Ginásio',\
                                   'Hospital': 'Zona Proximidade - Hospital',\
                                   'Jardins': 'Zona Proximidade - Jardins',\
                                   'Jardins Infância': 'Zona Proximidade - Jardins Infância',\
                                   'Padaria': 'Zona Proximidade - Padaria',\
                                   'Supermercado': 'Zona Proximidade - Supermercado',\
                                   'Bancos': 'Zona Proximidade - Bancos',\
                                   'Farmácia': 'Zona Proximidade - Farmácia',\
                                   'Praias': 'Zona Proximidade - Praias',\
                                   'Serviços Públicos': 'Zona Proximidade - Serviços Públicos',\
                                   'Bombeiros': 'Zona Proximidade - Bombeiros',\
                                   'Clínica': 'Zona Proximidade - Clínica',\
                                   'Polícia': 'Zona Proximidade - Polícia'})
        
data_era_moradias.head()